# Import RPB model along with other utility functions

In [1]:
from RPB_model import *

expression symbols from pyomo.core.expr  (deprecated in 6.6.2.dev0) (called
from <frozen importlib._bootstrap>:241)


# Creating model for a single side/section of the RPB

In [ ]:
# Create model instance. Currently, mode can be either "adsorption" or "desorption" which sets the boundary conditions for each case.
# m=RPB_model(mode="adsorption")
# or
m=RPB_model(mode="desorption")

# Custom initialization routine. Uses a homotopy routine to adjust the initialization factors so that all equations are 'active'.
single_section_init(m)

In [ ]:
# Plotting the results/profiles

plotting(m)

In [ ]:
# Some various utility functions that I have been using to check model performance
evaluate_MB_error(m)

print(f'CO2 Capture = {m.CO2_capture():.3}')

# check_scaling(m)

# degen_hunter(m)

# jac, variables, constraints = scaling_script(m)

# Creating a full RPB model

In [2]:
RPB = full_model_creation(lean_temp_connection=True)
init_routine_1(RPB)
fix_capture_and_solve(RPB, capture=0.9)

Ipopt 3.13.2: max_iter=500


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collection of Fortran codes for large-scale scientific
        computation

In [3]:
report(RPB)

,Value,Doc,Fixed
ads.L,3.000000,Bed Length [m],True
ads.D,10.000000,Bed diameter [m],True
ads.w_rpm,1.000000,bed rotational speed [revolutions/min],True
ads.theta,0.500000,Fraction of bed [-],True
des.theta,0.500000,Fraction of bed [-],False
ads.P_in,1.020098,Inlet flue gas pressure [bar],False
ads.P_out,1.013250,Outlet adsorber pressure [bar],True
ads.F_in,48.571329,Inlet adsorber gas flow [mol/s],False
ads.Tg_in,363.000000,Inlet flue gas temperature [K],True
ads.Tx,363.000000,"heat exchange fluid temperature, constant [K]",True


In [5]:
evaluate_MB_error(RPB.des)

N2 error = -3.33e-14 %
CO2 error = -0.324 %
H2O error = -3.33e-14 %


In [6]:
check_scaling(RPB)

Jacobian Condition Number: 8.52e+08


# Save and load model

In [7]:
# save model
to_json(RPB, fname="reduced node number base case.json.gz", gz=True, human_read=False)

In [46]:
# create model, load, and solve
RPB = full_model_creation(lean_temp_connection=True)
fix_capture(RPB, capture = 0.9)

from_json(RPB, fname="reduced node number base case.json.gz", gz=True)

solver = SolverFactory("ipopt")
solver.options = {
    "max_iter": 1000,
    "bound_push": 1e-22,
    # "halt_on_ampl_error": "yes",
}
solver.solve(RPB, tee=True).write()

Ipopt 3.13.2: max_iter=1000
bound_push=1e-22


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collection of Fortran codes for large-scale scientific
 

Optimization Testing

In [33]:
# create regularization parameter for the objective function
RPB.alpha_obj = Param(initialize=0.5, mutable=True)

# add objective
@RPB.Expression()
def obj(RPB):
    return RPB.alpha_obj * RPB.energy_requirement/250 - (1 - RPB.alpha_obj) * RPB.productivity/5

RPB.objective = Objective(expr=RPB.obj)

In [34]:
RPB.ads.L.setub(10)
RPB.ads.L.pprint()

L : Bed Length [m]
    Size=1, Index=None, Units=m
    Key  : Lower : Value : Upper : Fixed : Stale : Domain
    None :   0.1 :     3 :    10 :  True :  True :  Reals


In [35]:
RPB.ads.Tx.setlb(25+273)
RPB.ads.Tx.setub(95+273)
RPB.ads.Tx.pprint()

Tx : heat exchange fluid temperature, constant [K]
    Size=1, Index=None, Units=K
    Key  : Lower : Value : Upper : Fixed : Stale : Domain
    None :   298 :   363 :   368 :  True :  True :  Reals


In [36]:
RPB.des.Tx.setlb(100+273)
RPB.des.Tx.setub(160+273)
RPB.des.Tx.pprint()

Tx : heat exchange fluid temperature, constant [K]
    Size=1, Index=None, Units=K
    Key  : Lower : Value : Upper : Fixed : Stale : Domain
    None :   373 :   393 :   433 :  True :  True :  Reals


In [37]:
RPB.des.P_in.setub(1.1)
RPB.des.P_in.pprint()

P_in : Inlet flue gas pressure [bar]
    Size=1, Index=None, Units=bar
    Key  : Lower : Value : Upper : Fixed : Stale : Domain
    None :     1 :   1.1 :   1.1 :  True :  True :  Reals


In [38]:
degrees_of_freedom(RPB)

0

In [43]:
# free up decision variables
RPB.ads.L.unfix()
RPB.ads.theta.unfix()
# RPB.ads.w_rpm.unfix()
RPB.des.P_in.unfix()
# RPB.ads.Tx.unfix()
# RPB.des.Tx.unfix()

In [44]:
degrees_of_freedom(RPB)

3

In [45]:
# solve
solver = SolverFactory("ipopt")
solver.options = {
    "max_iter": 1000,
    "bound_push": 1e-15,
    # "halt_on_ampl_error": "yes",
    "tol": 1e-4,
    "max_cpu_time": 5*60,
    # "mu_strategy": "adaptive",
}
solver.solve(RPB, tee=True).write()

Ipopt 3.13.2: max_iter=1000
bound_push=1e-15
tol=0.0001
max_cpu_time=300


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collection of Fortran codes 

In [42]:
report(RPB)

,Value,Doc,Fixed
ads.L,3.078282,Bed Length [m],False
ads.D,10.000000,Bed diameter [m],True
ads.w_rpm,1.000000,bed rotational speed [revolutions/min],True
ads.theta,0.500000,Fraction of bed [-],True
des.theta,0.500000,Fraction of bed [-],False
ads.P_in,1.017944,Inlet flue gas pressure [bar],False
ads.P_out,1.013250,Outlet adsorber pressure [bar],True
ads.F_in,32.953814,Inlet adsorber gas flow [mol/s],False
ads.Tg_in,363.000000,Inlet flue gas temperature [K],True
ads.Tx,363.000000,"heat exchange fluid temperature, constant [K]",True


In [ ]:
[RPB.ads.qCO2_eq[z,o]() for z,o in RPB.ads.qCO2_eq.keys()]

In [ ]:
RPB.ads.Tx()

In [ ]:
full_contactor_plotting(RPB)

In [ ]:
get_init_factors(RPB.des)